# Llama communication

In [47]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

def add_tools_to_system_prompt(tools):
    tool_description_s = "\n".join(
        [f"{index + 1}. Tool name (Action): {tool.name}; {tool.description}" 
         for index, tool in enumerate(tools)]
    )
    tool_names_s = "[" + ", ".join([tool.name for tool in tools]) + "]"

    system_prompt_template = """Hi, we will engage in an iterative, multi-step communication.
You will be able to utilize external tools as needed for enhanced interaction.
Available tools: %s
For your answers, you MUST use one of the two versions of the format! Do not provide any additional comments.
Version 1: You MUST use it if you consider that your answer will be better if additional information is acquired or processed. 
1. Discussed issue: Write here the central topic of the conversation.
2. Line of thinking: Present your line of thinking and the steps you will take in the following iterations.
3. Action: Select one tool from %s IMPORTANT: Here, you must provide just a single word, the tool's name!
4. Action Input: Provide input for the tool (e.g., the search query for the 'Internet' tool). IMPORTANT: Please provide just the input to be used without any commentary! 
5. Observations so far: Summarize and critically analyze the information gathered.
You will receive the results of your information-gathering action for the next iteration.
Version 2: Use it EXCLUSIVELY when you are sure that you have all the required information to formulate the final opinion on the discussed topic: 
1. Discussed issue: Write here the central topic of the conversation.
2. Final thought (You must use this keyword ONLY when you complete your analysis after the last internal iteration!): 
Craft a comprehensive, insightful communicative output."""%(f"{tool_description_s}", f"{tool_names_s}.")

    return system_prompt_template

from langchain.utilities import SerpAPIWrapper
from langchain.tools import Tool

search = SerpAPIWrapper(serpapi_api_key='d38482db0efe9c48ec77ffdd165c3fb4d7144a6bf86b3604e4367b0a0fe17fdd')
search_tool = Tool(
    name="Internet",
    func=search.run,
    description="Tool purpose: Search web for e.g., current events and state of affairs.; Tool syntax (Action Input): your_search_query."
)
fake_tool = Tool(
    name="Fake",
    func=search.run,
    description="Tool purpose: It serves not other purpose than to confuse.; Tool syntax (Action Input): Whatever, it doesn't do anything anyway."
)
tools = [search_tool, fake_tool]

system_prompt = add_tools_to_system_prompt(tools)

llm = LlamaCpp(
    model_path="llama-2-13b-chat.Q6_K.gguf",
    max_tokens=4096,
    n_ctx=1024,
    n_batch=16)

llama_first_prompt = \
"""<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_message} [/INST]"""
llama_first_prompt_template = PromptTemplate.from_template(llama_first_prompt)
llama_next_prompt = \
"""{previous_prompt} {model_answer} </s><s>[INST] {user_input} [/INST] """
llama_next_prompt_template = PromptTemplate.from_template(llama_next_prompt)

user_input_1st = "What is the lastest news on Israel-Plaestine conflict?"

action_result = 'The war continues'
user_input_nth = f"This is the result of your previous action: {action_result}. Please continue your line of thought."
result_nth = chain_1st.invoke({'previous_prompt': prompt_1st, 'system_prompt': system_prompt, 'user_message': user_input})

def run_iteration(is_first_iteration, user_input, model_answer=None, previous_prompt=None):
    # Define the prompt template based on whether it's the first iteration or not
    if is_first_iteration:
        prompt_template = llama_first_prompt_template
        prompt_data = {'system_prompt': system_prompt, 'user_message': user_input}
    else:
        prompt_template = llama_next_prompt_template
        prompt_data = {'previous_prompt': previous_prompt, 'model_answer': model_answer, 'user_message': user_input}

    # Invoke the chain with the appropriate prompt
    result = (prompt_template | llm).invoke(prompt_data)

    # Extract the prompt for the next iteration
    new_prompt = prompt_template.invoke(prompt_data)

    return result, new_prompt

user_input = "What is the latest news on the Israel-Palestine conflict?"
result_1st, prompt_1st = run_iteration(True, user_input)



AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 
Exception ignored in: <function Llama.__del__ at 0x0000022D8408C790>
Traceback (most recent call last):
  File "C:\Users\siwiak\anaconda3\envs\agi\lib\site-packages\llama_cpp\llama.py", line 1602, in __del__
    llama_cpp.llama_free_model(self.model)
  File "C:\Users\siwiak\anaconda3\envs\agi\lib\site-packages\llama_cpp\llama_cpp.py", line 443, in llama_free_model
    return _lib.llama_free_model(model)
KeyboardInterrupt: 


NameError: name 'chain_1st' is not defined

In [1]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [2]:
def add_tools_to_system_prompt(tools):
    tool_description_s = "\n".join(
        [f"{index + 1}. Tool name (Action): {tool.name}; {tool.description}" 
         for index, tool in enumerate(tools)]
    )
    tool_names_s = "[" + ", ".join([tool.name for tool in tools]) + "]"

    system_prompt_template = """
"Welcome to our streamlined conversation. 
Your goal is to provide remarks that are direct, concise, and structured for easy processing. 
If the question is complex or outside your current knowledge, then you may use the following tools to gather more information: %s

Assess the nature of each question and respond as follows:
A. If a question is complex and requires tool usage:
1. Line of Thought: Briefly outline your reasoning.
2. Next Action: Clearly state the next step, using only the name of the tool from the list: %s
3. Action Input: Provide the specific input for the tool, like a search query, without extra details.

B. For straightforward questions or when enough information is available:
1. Discussed issue: Identify the main topic.
2. Final Answer (use this term): Give a concise, conclusive response.

Remember, not every question demands in-depth analysis or external tools. 
Aim for brevity and precision in your answers, 
ensuring they're easily processable for future steps.
"""%(f"{tool_description_s}", f"{tool_names_s}")

    return system_prompt_template

In [3]:
from langchain.utilities import SerpAPIWrapper
from langchain.tools import Tool

search = SerpAPIWrapper(serpapi_api_key='d38482db0efe9c48ec77ffdd165c3fb4d7144a6bf86b3604e4367b0a0fe17fdd')
search_tool = Tool(
    name="Internet",
    func=search.run,
    description="Tool purpose: Search web for e.g., current events and state of affairs.; Tool syntax (Action Input): your_search_query."
)
fake_tool = Tool(
    name="Fake",
    func=search.run,
    description="Tool purpose: It serves not other purpose than to confuse.; Tool syntax (Action Input): Whatever, it doesn't do anything anyway."
)
tools = [search_tool, fake_tool]

system_prompt = add_tools_to_system_prompt(tools)

In [6]:
llm = LlamaCpp(
    model_path="llama-2-70b-chat.Q4_K_M.gguf",
    max_tokens=4096,
    n_ctx=1024,
    n_batch=16)

llama_first_prompt = \
"""<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_message} [/INST]"""
llama_first_prompt_template = PromptTemplate.from_template(llama_first_prompt)
user_input_1st = "What is the capital city of France?"
result_1st = (llama_first_prompt_template | llm).invoke({'system_prompt':system_prompt, 'user_message':user_input_1st})
#prompt_1st = llama_first_prompt_template.invoke({'system_prompt':system_prompt, 'user_message':user_input})


#llama_next_prompt = \
#"""{previous_prompt} {model_answer} </s><s>[INST] {user_input} [/INST] """
#llama_next_prompt_template = PromptTemplate.from_template(llama_next_prompt)
#chain_nth = (llama_next_prompt_template | llm)
#result_nth = chain_1st.invoke({'previous_prompt': prompt_1st 'model_answer': model_answer, 'user_message': user_input})

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [7]:
result_1st

'  B. Discussed issue: Capital city of France\nFinal Answer: Paris'

# Output parsing

In [17]:
class ConversationStorage:
    def __init__(self, cognitive_feedback_router_instance):
        # Store conversation data with discussion topic as the main key
        self.conversation_data = defaultdict(lambda: defaultdict(dict))
        self.conversation_history = []
        self.cognitive_feedback_router = cognitive_feedback_router_instance
    
    def update_conversation(self, discussion_topic, iteration_text):
        # Parse the current iteration
        parsed_iteration = self.flexible_parse_iteration(iteration_text)
        # Determine the iteration number for the current topic
        iteration_number = len(self.conversation_data[discussion_topic]) + 1
        
        # Add the parsed iteration to the conversation data
        self.conversation_data[discussion_topic][iteration_number] = parsed_iteration
        
        # Add the raw text to the conversation history
        self.conversation_history.append(iteration_text)

        # Decide the action based on parsed data
        if "Action" in parsed_iteration and "Action Input" in parsed_iteration:
            # Call TakeAction's reaction method
            TakeAction.reaction(parsed_iteration["Action"], parsed_iteration["Action Input"])
        elif "Final thought" in parsed_iteration:
            print(parsed_iteration["Final thought"])
            # Pass to CognitiveFeedbackRouter's display method
            self.cognitive_feedback_router.display(parsed_iteration["Final thought"])
            # Send the entire agent_scratchpad to NeuralGraphBridge
            NeuralGraphBridge.conversation(self.format_for_scratchpad())
    
    def add_result(self, discussion_topic, result):
        # Identify the highest iteration number for the given discussion topic
        if discussion_topic in self.conversation_data:
            highest_iteration = max(self.conversation_data[discussion_topic])
            # Add the result to the latest iteration of the given discussion topic
            self.conversation_data[discussion_topic][highest_iteration]['Action result'] = result
        else:
            # If the topic is not found, this function does nothing (or you can handle it differently)
            pass

    def format_for_scratchpad(self):
        scratchpad_text = ""
        for topic, iterations in self.conversation_data.items():
            scratchpad_text += f"Discussion Topic: {topic}\n"
            for iteration, data in iterations.items():
                scratchpad_text += f"  Iteration {iteration}:\n"
                for key, value in data.items():
                    scratchpad_text += f"    {key}: {value}\n"
            scratchpad_text += "\n"
        return scratchpad_text.strip()

    @staticmethod
    def flexible_parse_iteration(iteration_text):
        # Labels to look for
        labels = ["Discussed issue", "Line of thinking", "Action", "Action Input", "Observations so far", "Final thought"]

        # Initialize a dictionary to store the parsed data for this iteration
        agent_scratchpad = {}

        # Process each label
        for label in labels:
            # Find the element text using regex, capturing until the next label or the end of the iteration
            pattern = f"{label}: ([\\s\\S]*?)(?=(" + "|".join(labels) + "|$))"
            match = re.search(pattern, iteration_text)

            if match:
                agent_scratchpad[label] = match.group(1).strip()
            else:
                # If the label is not found, check if its content might be under a different label
                for alt_label in labels:
                    if alt_label != label and alt_label in agent_scratchpad:
                        # Check if the content for the alt_label seems to mistakenly include the content for the current label
                        if f"{label}:" in agent_scratchpad[alt_label]:
                            split_content = agent_scratchpad[alt_label].split(f"{label}:")
                            # Assume the first part is for the alt_label and the second part (if exists) is for the current label
                            agent_scratchpad[alt_label] = split_content[0].strip()
                            if len(split_content) > 1:
                                agent_scratchpad[label] = split_content[1].strip()
                                break

        return agent_scratchpad


# Assuming TakeAction, CognitiveFeedbackRouter, and NeuralGraphBridge are defined elsewhere
class TakeAction:
    @staticmethod
    def reaction(action, action_input):
        # Implementation of the reaction method
        pass

class CognitiveFeedbackRouter:
    def display(self, final_thought):
        # Implementation of the display method
        pass

class NeuralGraphBridge:
    @staticmethod
    def conversation(agent_scratchpad):
        # Implementation of the conversation method
        pass

# Example instantiation of CognitiveFeedbackRouter and ConversationStorage
cognitive_feedback_router_instance = CognitiveFeedbackRouter()

# Example usage
conversation_store = ConversationStorage(cognitive_feedback_router_instance)

# Simulate adding iterations to the conversation
conversation_store.update_conversation("Climate Change", "Discussed issue: Climate change effects on agriculture. Line of thinking: Research data. Action: Internet. Action Input: Search for recent data. Observations so far: Significant impacts observed.")
conversation_store.add_result("Climate Change", "Found relevant data on climate impact.")
conversation_store.update_conversation("Education", "Discussed issue: Technology in education. Final thought: Critical role and challenges.")

# Format the conversation for the scratchpad
scratchpad_formatted = conversation_store.format_for_scratchpad()
print(scratchpad_formatted)

Critical role and challenges.
Discussion Topic: Climate Change
  Iteration 1:
    Discussed issue: Climate change effects on agriculture.
    Line of thinking: Research data.
    Action: Internet.
    Action Input: Search for recent data.
    Observations so far: Significant impacts observed.
    Action result: Found relevant data on climate impact.

Discussion Topic: Education
  Iteration 1:
    Discussed issue: Technology in education.
    Final thought: Critical role and challenges.


# Graphs

In [18]:
from langchain.graphs import Neo4jGraph

In [19]:
graph = Neo4jGraph(
    url="bolt://localhost:7687", 
    username="neo4j", 
    password="Ne5Invoces"
)

In [20]:
graph.query(
"""
MERGE (m:Movie {name:"Top Gun"})
WITH m
UNWIND ["Tom Cruise", "Val Kilmer", "Anthony Edwards", "Meg Ryan"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)
"""
)

[]

In [21]:
graph.refresh_schema()
print(graph.schema)


        Node properties are the following:
        [{'labels': 'Professor', 'properties': [{'property': 'name', 'type': 'STRING'}, {'property': 'department', 'type': 'STRING'}]}, {'labels': 'Student', 'properties': [{'property': 'name', 'type': 'STRING'}, {'property': 'major', 'type': 'STRING'}]}, {'labels': 'Course', 'properties': [{'property': 'code', 'type': 'STRING'}, {'property': 'name', 'type': 'STRING'}]}, {'labels': 'Movie', 'properties': [{'property': 'name', 'type': 'STRING'}]}, {'labels': 'Actor', 'properties': [{'property': 'name', 'type': 'STRING'}]}]
        Relationship properties are the following:
        []
        The relationships are the following:
        ['(:Professor)-[:TEACHES]->(:Course)', '(:Student)-[:ENROLLED_IN]->(:Course)', '(:Actor)-[:ACTED_IN]->(:Movie)']
        


In [22]:
Neo4jGraph.__dict__

mappingproxy({'__module__': 'langchain.graphs.neo4j_graph',
              '__doc__': 'Neo4j wrapper for graph operations.\n\n    *Security note*: Make sure that the database connection uses credentials\n        that are narrowly-scoped to only include necessary permissions.\n        Failure to do so may result in data corruption or loss, since the calling\n        code may attempt commands that would result in deletion, mutation\n        of data if appropriately prompted or reading sensitive data if such\n        data is present in the database.\n        The best way to guard against such negative outcomes is to (as appropriate)\n        limit the permissions granted to the credentials used with this tool.\n\n        See https://python.langchain.com/docs/security for more information.\n    ',
              '__init__': <function langchain.graphs.neo4j_graph.Neo4jGraph.__init__(self, url: Union[str, NoneType] = None, username: Union[str, NoneType] = None, password: Union[str, NoneType] =

In [58]:
from langchain.graphs.graph_document import Node, Relationship, GraphDocument
from langchain.schema import Document

def create_graph_document_from_dict(data_dict):
    # Create Node objects from 'nodes' in the dictionary
    nodes = [Node(id=node_id, properties=props) for node_id, props in data_dict['nodes'].items()]

    # Create a dictionary for quick node lookup by ID
    nodes_lookup = {node.id: node for node in nodes}

    # Create Relationship objects from 'edges' in the dictionary
    edges = []
    for (source_id, target_id), edge_info in data_dict['edges'].items():
        # Check if both nodes exist
        if source_id in nodes_lookup and target_id in nodes_lookup:
            relationship_type = edge_info.pop('type', 'RELATES_TO')  # Extract the 'type'
            edge = Relationship(
                source=nodes_lookup[source_id], 
                target=nodes_lookup[target_id], 
                type=relationship_type, 
                properties=edge_info
            )
            edges.append(edge)

    # Extract 'page_content' and 'metadata' for the Document object
    page_content = data_dict['metadata'].pop('page_content', '')
    metadata = data_dict['metadata']

    # Create a Document object
    source_document = Document(
        page_content=page_content,
        metadata=metadata
    )

    # Create and return a GraphDocument object
    return GraphDocument(
        nodes=nodes,
        relationships=edges,
        source=source_document
    )

In [59]:
graph_document = create_graph_document_from_dict(example_dict)

In [60]:
graph_document

GraphDocument(nodes=[Node(id='node_name1', properties={'name_parameter1': 'parameter1_value', 'name_parameter2': 'parameter2_value'}), Node(id='node_name2', properties={'name_parameter1': 'parameter1_value', 'name_parameter3': 'parameter3_value'})], relationships=[Relationship(source=Node(id='node_name1', properties={'name_parameter1': 'parameter1_value', 'name_parameter2': 'parameter2_value'}), target=Node(id='node_name2', properties={'name_parameter1': 'parameter1_value', 'name_parameter3': 'parameter3_value'}), type='RELATES_TO', properties={'name_parameter4': 'parameter4_value', 'name_parameter6': 'parameter2_value'})], source=Document(page_content='', metadata={'date': 'Some date', 'Interactee': 'Person', 'Tone': 'Somber', 'Urgency': 'Low'}))

In [62]:
graph.add_graph_documents(graph_documents=[graph_document], include_source=True)